In [38]:
import pandas as pd

# Exemplo: se o arquivo estiver na mesma pasta do notebook
df = pd.read_csv("base.csv")

In [ ]:
%pip install pandas numpy scikit-learn xgboost catboost imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


# 🤖 MODELOS DE MACHINE LEARNING PARA PREVISÃO DE ATTRITION

In [40]:
# 1. Importando bibliotecas
%pip install imbalanced-learn

import pandas as pd
import numpy as np
import sklearn

print("✅ scikit-learn:", sklearn.__version__)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


Note: you may need to restart the kernel to use updated packages.
✅ scikit-learn: 1.7.0


In [41]:
import pandas as pd

# Exemplo: se o arquivo estiver na mesma pasta do notebook
df = pd.read_csv("base.csv")

In [42]:
display(df.head())

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [43]:
# 3. Criação das features engenheiradas (baseado no notebook anterior)
df['YearsInCurrentRoleRatio'] = df['YearsInCurrentRole'] / (df['YearsAtCompany'] + 1)
df['ExperiencePerAge'] = (df['TotalWorkingYears'] + 1) / (df['Age'] + 1)
df['OverallSatisfaction'] = df[['JobSatisfaction', 'EnvironmentSatisfaction', 'RelationshipSatisfaction']].mean(axis=1)
df['SatisfactionPerformanceGap'] = df['OverallSatisfaction'] - df['PerformanceRating']
df['YearsInCompanyToAge'] = df['YearsAtCompany'] / (df['Age'] + 1)
df['YearsSincePromotionRatio'] = df['YearsSinceLastPromotion'] / (df['YearsAtCompany'] + 1)
df['IsManager'] = df['JobRole'].isin(['Manager', 'Director']).astype(int)
df['TravelHighFreq'] = (df['BusinessTravel'] == 'Travel_Frequently').astype(int)
df['IsSingleOverTime'] = ((df['MaritalStatus'] == 'Single') & (df['OverTime'] == 'Yes')).astype(int)

In [44]:
# 4. Codificação da variável alvo
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

In [45]:
# 5. Codificação de variáveis categóricas com One-Hot Encoding
cat_cols = ['Gender', 'OverTime', 'MaritalStatus', 'BusinessTravel', 'Department', 'JobRole', 'EducationField']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

In [46]:
# 6. Seleção de variáveis (remoção de colunas irrelevantes)
X = df.drop(columns=['Attrition', 'EmployeeNumber', 'EmployeeCount', 'Over18', 'StandardHours'])
y = df['Attrition']


In [47]:
# 7. Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

In [48]:
# normalizando dados
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
%pip install scikit-learn==1.3.2 imbalanced-learn==0.11.0

from imblearn.over_sampling import SMOTE

# 8. Tratamento do desbalanceamento com SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_res, y_train)

  Using cached scikit_learn-1.7.0-cp312-cp312-win_amd64.whl.metadata (14 kB)
Note: you may need to restart the kernel to use updated packages.


ImportError: cannot import name '_deprecate_Xt_in_inverse_transform' from 'sklearn.utils.deprecation' (c:\Users\bgbir\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py)

##### ✔️ Justificativa: A base tem ~16% de casos positivos (Attrition = Yes), por isso aplicamos SMOTE apenas no treino para evitar vazamento de dados e simular a distribuição natural no teste.

In [50]:
# 9. Treinamento de 4 modelos distintos
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42), # 100 numero de arvores
    "XGBoost": XGBClassifier(eval_metric='logloss'),
    "CatBoost": CatBoostClassifier(verbose=0)
}

results = {}

for name, model in models.items():
    model.fit(X_train_res, y_train)
    y_pred = model.predict(X_test)

    # Armazenar métricas
    results[name] = {
        "F1-score": f1_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred)
    }


In [51]:
# 10. Comparação final dos modelos
# Trocando o F1-score por "Recall" na ordenação
resultados_df = pd.DataFrame(results).T.sort_values(by="Recall", ascending=False)
print("🏆 Comparativo de desempenho dos modelos:")
print(resultados_df)

🏆 Comparativo de desempenho dos modelos:
                     F1-score  Precision    Recall
Logistic Regression  0.513274   0.690476  0.408451
XGBoost              0.294118   0.483871  0.211268
CatBoost             0.319149   0.652174  0.211268
Random Forest        0.234043   0.478261  0.154930


🏆 Melhor modelo: Logistic Regression

**Justificativas:** <br>
<b>1 - Maior F1-score (0.4787):</b>

* O F1-score é a média harmônica entre Precisão e Recall, e é ideal quando você quer equilibrar ambos, especialmente em problemas com classes desbalanceadas. Isso indica que o modelo está conseguindo um bom equilíbrio entre acertar positivos e evitar falsos positivos.

**2 - Maior Recall (0.6318):**

* Isso significa que o modelo está detectando uma grande proporção dos verdadeiros positivos — importante em contextos onde não perder casos positivos é crítico (ex: fraudes, diagnósticos, etc.).